In [ ]:
import this

In [4]:
import nltk 
from nltk.corpus import state_union , stopwords ,movie_reviews


# Compiling Data

In [ ]:
#all the words in the corpus satate union
words = [w for w in state_union.words() if w.isalpha()]
print(len(words))
#remove all stop words
stopList = stopwords.words("english")
#eliminate stop words from our words list 
words = [w for w in words if w.lower() not in stopList]
print(len(words))

# Creating Frequency Distributions

In [ ]:
#analyse frequentiel
fd = nltk.FreqDist(words)
fd.tabulate(5)
fd.most_common(5)

#how many times a specific word is shown :
print(f" children : {fd['children']}\n child :  {fd['child']} \n kids : {fd['kids']}  \n kid : {fd['kid']}")

#to make a better analysis put all the words in lowercase
lowWords = [w.lower() for w in words]
fdLow = nltk.FreqDist(lowWords)
fdLow.tabulate(5)

# Extracting Concordance and Collocations
  * Where each occurrence appears
  * What words surround each occurrence

In [ ]:
text = nltk.Text(words)
 #it can only print 
# to save it into a variable 
print("--------------------------------------------------------------------------------")
lines = text.concordance_list("america", lines=2) # returns a list of ConcordanceLine objects
for line in lines :
    print(line.line)

Collocations are series of words that frequently appear together in a given text.
 * **Bigrams**: Frequent two-word combinations
 * **Trigrams**: Frequent three-word combinations
 * **Quadgrams**: Frequent four-word combinations

In [ ]:
test = [w for w in nltk.corpus.state_union.words() if w.isalpha()]
finder = nltk.collocations.TrigramCollocationFinder.from_words(test)
finder.ngram_fd.tabulate(2)



# Pre-Trained Sentiment Analyzer
VADER (Valence Aware Dictionary and sEntiment Reasoner).

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [13]:
sia = SentimentIntensityAnalyzer()
s1 = sia.polarity_scores("Oh God , i am tired af")
s2 = sia.polarity_scores("just stop living")
s3 = sia.polarity_scores("try this next time")
print(f"Oh God , i am tired af : {s1}")
print(f"just stop living : {s2}")
print(f"try this next time : {s3}")


Oh God , i am tired af : {'neg': 0.362, 'neu': 0.375, 'pos': 0.263, 'compound': -0.2023}
just stop living : {'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'compound': -0.296}
try this next time : {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [9]:
tweets = [t.replace("://", "//") for t in nltk.corpus.twitter_samples.strings()]


In [14]:
from random import shuffle

def is_positive_compound(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

def is_positive_pos(tweet: str) -> bool:
    """True if tweet has pos sentiment greater than half, False otherwise."""
    return sia.polarity_scores(tweet)["pos"] > 0.5
shuffle(tweets)
for tweet in tweets[:5]:
    print(">", is_positive_compound(tweet), is_positive_pos(tweet) )
    print(tweet)

> False False
penge bestfriend :( #PSYGustoKita
> False False
RT @Uptotherewithit: Farage just treated us to a master class in how to take a question and answer session. #UKIP
> True False
RT @SheldanKeay: I've watched this vine of Ed Miliband and The Rock ten times and I'm still laughing. https//t.co/T2MgBsObQY
> False False
@KatieKhaleesi I think what will happen is that Miliband becomes PM and then can't do anything for 5 years because he won't do deals w/ SNP.
> True False
RT @jamesdoleman: Sun sacks worm after it shows Ed Miliband won debate 

 http//t.co/9phEKZMiIq


In [ ]:
idFile = movie_reviews.fileids()[5]
#movie_reviews.raw(idFile)

In [5]:
positive_review_ids = movie_reviews.fileids(categories=["pos"])
negative_review_ids = movie_reviews.fileids(categories=["neg"])
all_review_ids = positive_review_ids + negative_review_ids


In [6]:
from statistics import mean

def is_positive_review(review_id: str , pos = False) -> bool:
    if pos == False :
        """True if the average of all sentence compound scores is positive."""
        text = nltk.corpus.movie_reviews.raw(review_id)
        scores = [
            sia.polarity_scores(sentence)["compound"]
            for sentence in nltk.sent_tokenize(text)
        ]
        return mean(scores) > 0
    else :
        """True if tweet has pos sentiment greater than half, False otherwise."""
        text = nltk.corpus.movie_reviews.raw(review_id)
        scores = [
            sia.polarity_scores(sentence)["pos"]
            for sentence in nltk.sent_tokenize(text)
        ]
        return mean(scores) > 0.5


In [15]:
from tqdm.notebook import tqdm_notebook
shuffle(all_review_ids)
correct = 0
for review_id in all_review_ids:
     if is_positive_review(review_id):
        if review_id in positive_review_ids:
             correct += 1
        else:
            if review_id in negative_review_ids:
                correct += 1

print(F"{correct / len(all_review_ids):.2%} correct")


69.10% correct


In [ ]:
# this cel is jst to understand pos_tag
with open("zen.txt",'r') as f :
    text = f.read()
clean_words = [word for word, tag in tqdm_notebook( filter(
    skip_unwanted,
    nltk.pos_tag(nltk.word_tokenize(text)))
)]

#nltk.pos_tag(nltk.word_tokenize(text ),  tagset='universal')

# Customizing NLTK’s Sentiment Analysis
 Begin by excluding unwanted words and building the initial category groups:

In [16]:
# create a list of unwanted words  = stopWords + Names
from tqdm.notebook import tqdm_notebook

unwanted = nltk.corpus.stopwords.words("english")
unwanted.extend([w.lower() for w in nltk.corpus.names.words()])

def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in unwanted:
        return False
    if tag.startswith("NN"):
        return False
    return True

positive_words = [word for word, tag in tqdm_notebook( filter(
    skip_unwanted,
    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["pos"])))
)]
negative_words = [word for word, tag in tqdm_notebook(filter(
    skip_unwanted,
    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["neg"])))
)]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Finding the common set so you can remove it from the distribution objects

In [17]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}

Set up the positive and negative bigram finders

In [18]:
positive_bigram_finder = nltk.collocations.BigramCollocationFinder.from_words([
    w.lower() for w in nltk.corpus.movie_reviews.words(categories=["pos"])
    if w.isalpha() and w not in unwanted
])
negative_bigram_finder = nltk.collocations.BigramCollocationFinder.from_words([
    w for w in nltk.corpus.movie_reviews.words(categories=["neg"])
    if w.isalpha() and w not in unwanted
])

In [ ]:
positive_trigram_finder = nltk.collocations.TrigramCollocationFinder.from_words([
    w.lower() for w in nltk.corpus.movie_reviews.words(categories=["pos"])
    if w.isalpha() and w not in unwanted
])
negative_trigram_finder = nltk.collocations.TrigramCollocationFinder.from_words([
    w for w in nltk.corpus.movie_reviews.words(categories=["neg"])
    if w.isalpha() and w not in unwanted
])

# Training and Using a Classifier


In [19]:
def mean(lst):
    return sum(lst)/len(lst)

def extract_features(text):
    features = dict()
    wordcount = 0
    compound_scores = list()
    positive_scores = list()

    for sentence in tqdm_notebook(nltk.sent_tokenize(text)):
        for word in nltk.word_tokenize(sentence):
            if word.lower() in top_100_positive:
                wordcount += 1
        compound_scores.append(sia.polarity_scores(sentence)["compound"])
        positive_scores.append(sia.polarity_scores(sentence)["pos"])

    # Adding 1 to the final compound score to always have positive numbers
    # since some classifiers you'll use later don't work with negative numbers.
    features["mean_compound"] = mean(compound_scores) + 1
    features["mean_positive"] = mean(positive_scores)
    features["wordcount"] = wordcount

    return features

In [20]:
features = [
    (extract_features(nltk.corpus.movie_reviews.raw(review)), "pos")
    for review in nltk.corpus.movie_reviews.fileids(categories=["pos"])
]
features.extend([
    (extract_features(nltk.corpus.movie_reviews.raw(review)), "neg")
    for review in nltk.corpus.movie_reviews.fileids(categories=["neg"])
])

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [21]:
from random import shuffle
# Use 1/4 of the set for training
train_count = len(features) // 4
shuffle(features)
classifier = nltk.NaiveBayesClassifier.train(features[:train_count])
classifier.show_most_informative_features(10)

Most Informative Features
               wordcount = 2                 pos : neg    =      3.0 : 1.0
               wordcount = 0                 neg : pos    =      1.7 : 1.0
               wordcount = 1                 pos : neg    =      1.6 : 1.0


In [22]:
nltk.classify.accuracy(classifier, features[train_count:])

0.648

In [33]:
with open("zen.txt" , 'r' , encoding='utf-8') as f:
    new_review = f.read()
new_review = nltk.Text(new_review)
classifier.classify(new_review)


AttributeError: 'Text' object has no attribute 'copy'

now we will try to eliminate the commun ngrams flowing the previous logic

In [ ]:
positive_fd = positive_bigram_finder.ngram_fd
negative_fd = negative_bigram_finder.ngram_fd

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive_bigram = {bigram for bigram, count in positive_fd.most_common(100)}
top_100_negative_bigram = {bigram for bigram, count in negative_fd.most_common(100)}



extracting features using bigrams instead of singl words

# Using scikit-learn Classifiers With NLTK


In [34]:
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [35]:
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

In [36]:
# Use 1/4 of the set for training
train_count = len(features) // 4
shuffle(features)
for name, sklearn_classifier in  tqdm_notebook(classifiers.items()):
    classifier = nltk.classify.SklearnClassifier(sklearn_classifier)
    classifier.train(features[:train_count])
    accuracy = nltk.classify.accuracy(classifier, features[train_count:])
    print(F"{accuracy:.2%} - {name}")

  0%|          | 0/9 [00:00<?, ?it/s]

67.40% - BernoulliNB
67.40% - ComplementNB
67.40% - MultinomialNB
71.33% - KNeighborsClassifier
64.47% - DecisionTreeClassifier
68.20% - RandomForestClassifier
73.13% - LogisticRegression
73.00% - MLPClassifier
70.13% - AdaBoostClassifier
